In [1]:
from Bio import SeqIO
from collections import Counter
import pandas as pd 
import re
import requests
import json
import time
import glob
import pprint
import seaborn as sns
import pylab as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import ListedColormap
import pycountry 
import pycountry_convert as pc
pp = pprint.PrettyPrinter()
import numpy as np
from tabulate import tabulate
import geopandas as gpd
from geopandas import GeoDataFrame
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
import regex as re
import plotly.graph_objects as go
import os

from string import punctuation
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
import nltk
from bs4 import BeautifulSoup


 

stemmer=SnowballStemmer("english")
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import networkx as nx
from collections import defaultdict
from itertools import combinations
#from fa2 import ForceAtlas2

import textmining
  

%env folder = X:\5_Research\Paul\dsi_origins\

env: folder=X:\5_Research\Paul\dsi_origins\


In [2]:
path = %env folder

In [ ]:
#create df
df_wildsi_country =pd.read_csv(path + R'processed_data\wildsi_country.csv', index_col = 0)

In [5]:
#standardize publication identifiers
pmids = list(df_wildsi_country['PRIMARY_PMID'])
pmcids = list(df_wildsi_country['PRIMARY_PMCID'])
dois = list(df_wildsi_country['PRIMARY_DOI'])


pmids_clean = []
for pubid in pmids: 
    if pd.isna(pubid): 
        pmids_clean.append(np.NaN)
    else: 
        if type(pubid) == str: 
            if ';' in pubid: 
                pmids_clean.append(float(pubid.split(';')[0])) #multiple pubmeds given, so only take first one
            else: 
                pmids_clean.append(float(pubid))
        elif type(pubid) == float or type(pubid) == int: 
                pmids_clean.append(float(pubid))
        else: 
            print(pubid)
            print(type(pubid))

pmcids_clean = []
for pmcid in pmcids: 
    if pd.notna(pmcid): 
        pmcids_clean.append(pmcid.lower())
    else: 
        pmcids_clean.append(np.nan)

dois_clean = []
for pub in list(dois):
    pub_ed = pub
    if pd.isna(pub_ed): 
        dois_clean.append(np.nan)
    else: 
        if 'DOI' in pub_ed:
            pub_ed = pub_ed.replace('DOI: ', '').replace('DOI:', '')
            dois_clean.append(pub_ed)
        elif ';' in pub_ed:
            pub_ed = pub_ed.split(';')[0]
            dois_clean.append(pub_ed)
        elif 'Online' in pub_ed: 
            pub_ed = pub_ed.replace('Online First', '')
            dois_clean.append(pub_ed)
        elif ' ' in pub_ed: 
            pub_ed = pub_ed.replace(' ','')
            dois_clean.append(pub_ed)
        else: 
            dois_clean.append(pub_ed)

In [6]:
df_wildsi_country['PRIMARY_PMID'] =pmids_clean
df_wildsi_country['PRIMARY_PMCID'] = pmcids_clean
df_wildsi_country['PRIMARY_DOI'] =dois_clean

In [9]:
for x in list(df_wildsi_country['PRIMARY_PMID'][df_wildsi_country['PRIMARY_PMID'].notna()]): 
    if type(x) != float: 
        print(type(x))
        

In [ ]:
# select ids working pub ids 
df_clean = df_wildsi_country[['ACCESSION', 'PRIMARY_PMID', 'PRIMARY_PMCID', 'PRIMARY_DOI']].dropna(subset= ['PRIMARY_PMID', 'PRIMARY_PMCID', 'PRIMARY_DOI'], how='all', inplace=False).drop_duplicates().reset_index(drop = True)
df_clean['id_select'] = df_clean[['PRIMARY_PMID', 'PRIMARY_PMCID', 'PRIMARY_DOI']].bfill(axis=1).iloc[:, 0]

In [11]:
df_clean[['ACCESSION','id_select']].to_csv(path+R'processed_data/acc_id.csv')

In [23]:
df_wildsi_country_idselect = pd.merge(df_wildsi_country, df_clean[['ACCESSION','id_select']], how = 'left')
df_wildsi_country_idselect.to_csv(path+R'processed_data/wildsi_country_idselect.csv')

In [33]:
#export pubid lists for downloading from the lens
publist_pmid = list(set(df_wildsi_country['PRIMARY_PMID'][df_wildsi_country['PRIMARY_PMID'].notna()]))
pd.DataFrame(publist_pmid, columns = ['PRIMARY_PMID']).to_csv(path+R'processed_data/clean_pmids.csv')

#only take pmcids if no pmids available
publist_pmcid = list(set(df_wildsi_country['PRIMARY_PMCID'][(df_wildsi_country['PRIMARY_PMCID'].notna()) & (df_wildsi_country['PRIMARY_PMID'].isna())]))
pd.DataFrame(publist_pmcid, columns = ['PRIMARY_PMCID']).to_csv(path+R'processed_data/clean_pmcids.csv')

#if no other are available, take DOIs
publist_doi = list(set(df_wildsi_country['PRIMARY_DOI'][(df_wildsi_country['PRIMARY_DOI'].notna()) & (df_wildsi_country['PRIMARY_PMID'].isna()) & (df_wildsi_country['PRIMARY_PMCID'].isna())]))
pd.DataFrame(publist_doi, columns = ['PRIMARY_DOI']).to_csv(path+R'processed_data/clean_dois.csv')

### overview

In [ ]:
#df_wildsi_country_idselect = pd.read_csv(path+R'processed_data/wildsi_country_idselect.csv', index_col = 0)

In [ ]:
#df_clean = pd.read_csv(path+R'processed_data/acc_id.csv', index_col = 0)

In [35]:
#nr of unique accession numbers with publication ID
len(set(df_clean['ACCESSION']))

12040057

In [37]:
#nr of unique publication IDs
len(set(df_clean['id_select']))

101111

In [39]:
len(publist_pmid)

81147

In [41]:
len(publist_pmcid)

17490

In [101]:
len(publist_doi)

2516